# Assignment 2:  solving puzzles with search


Joshua Burris

In this assignment we will solve a sliding block puzzle not unlike the Eight Puzzle that we have seen in class.  The puzzle is called "Making two L's meet".  It's played on a 5 x 5 grid with nine pieces that are placed as follows:![](https://www.cs.colostate.edu/~cs440/fall19/assignments/images/initial.png)

The pieces are numbered from 1 to 9, and the two L-shaped pieces are numbered 2 and 7.  The un-numbered regions are empty.  A piece can be moved into an adjacent empty space that can accommodate it.  For example, piece number 9 can be moved left or right, and piece number 7 can be moved down.  Note that pieces 1,2,7,8 take up three squares, 3,4,5,6 take up two squares, and 9 takes up a single square.  The following is an example of a valid solution to the puzzle:
![](https://www.cs.colostate.edu/~cs440/fall19/assignments/images/goal.png)

To write a program that solves the puzzle we need to define the possible actions at a given state.  Whereas for the Eight Puzzle, an action could be specified by the direction in which the empty square is moved, this is no longer possible in this puzzle.  Here an action is a tuple `(piece, direction)` where `piece` is the piece that needs to be moved, numbered between 1 and 9, and `direction` is one of `'UP', 'DOWN', 'LEFT', 'RIGHT'`.

As discussed in class, uninformed search can be costly to run.  Therefore we will focus on solving this problem using the A* algorithm.  For that, you will need to propose and implement heuristics that will assist the algorithm in finding the solution in a timely manner.

For your implementation use the code provide in `search.py`.  Using this framework, what you need to do is to write a class that inherits from the generic class `Problem`.  We'll call that class `two_Ls`.  A stub that you need to complete is provided below.  You will need to add methods that will allow you to solve the puzzle with the search algorithms implemented in `search.py`.  


In [33]:
import search
from search import Problem

moves = {'UP': -5, 'DOWN': 5, 'LEFT': -1, 'RIGHT': 1}

class two_Ls(Problem) :
    """
    A class that encapsulates the 'Making two Ls meet' puzzle
    """
    # fill in the necessary methods to make the class work in conjuntion
    # with search functions in search.py
    # this incl. methods such as actions, goal_test etc.
    # see the implementation of EightPuzzle in search.py for an example
    # you can follow
    def __init__(self, initial=(1,1,1,2,2, 3,3,4,4,2, 7,5,5,6,6, 7,7,8,8,8, 0,0,0,9,0)) :
        self.initial = initial
        #Problem.__init__(self, initial)
    
    def actions(self, state) :
        actions = []
        block_sizes = [3, 3, 2, 2, 2, 2, 3, 3, 1]
        for key, value in moves.items() :
            block_inc = [0]*9
            for i in range(len(state)) :
                if state[i] >= 1 :
                    if not is_oob(i, i+value) and (state[i+value] == 0 or state[i+value] == state[i]) :
                        block_inc[state[i]-1] += 1
                        if block_inc[state[i]-1] == block_sizes[state[i]-1] :
                            actions += [(state[i], key)]
        return actions
    
    def result(self, state, action) :
        new_state = list(state)
        changed = []
        for i in range(len(state)) :
            if state[i] == action[0] :
                if i not in changed :
                    new_state[i] = 0
                new_square = i+moves[action[1]]
                new_state[new_square] = state[i]
                changed.append(new_square)
        return tuple(new_state)
    
    def goal_test(self, state) :
        seven_index = state.index(7)
        if is_oob(seven_index, seven_index+moves['RIGHT']) :
            return False
        if state[seven_index+moves['RIGHT']] == 2 :
            return True
        return False
    
    def path_cost(self, c, state1, action, state2) :
        return c + 1
    
    def value(self, state) :
        return 5 - distance(state.index(7), state.index(2)) - 1

In the following two cells, implement and describe two admissible heuristics for the problem.  Explain why they are admissible.  Make sure that the two heuristics are distinct from each other and capture different notions of the idea of a "good search direction".

In [34]:
def h1(node) :
    return distance(node.state.index(7), node.state.index(2)) - 1

def h2(node) :
    twos = []
    sevens = []
    for i in range(len(node.state)) :
        if node.state[i] == 2 :
            twos.append(i)
        elif node.state[i] == 7 :
            sevens.append(i)
    
    r2 = get_row(twos[1]); r7 = get_row(sevens[1])
    start_row = min(r2, r7); end_row = max(r2, r7)
    c2 = get_column(twos[1]); c7 = get_column(sevens[1])
    start_col = min(c2, c7); end_col = max(c2, c7)
    
    sum = 0
    for i in range(len(node.state)) :
        r = get_row(i)
        c = get_column(i)
        if r >= start_row and r <= end_row and c >= start_col and c <= end_col :
            s = node.state[i]
            if s != 2 and s != 7 and s != 0 :
                sum += 1
    return sum

def max_h(node) :
    return max(h1(node), h2(node))

def is_oob(start, end) :
    if end < 0 :
        return True
    if end >= 25 :
        return True
    if (start % 5) == 0 and end == (start-1) :
        return True
    if ((start+1) % 5) == 0 and end == (start+1) :
        return True
    return False

def distance(start, end) :
    return abs(get_column(end) - get_column(start)) + abs(get_row(end) - get_row(start))

def get_column(i) :
    return i % 5

def get_row(i) :
    row = 0
    tmp = i
    for j in range(5) :
        if tmp < 5 :
            return row
        tmp -= 5
        row += 1

def print_state(state) :
    for i in range(5) :
        s = "|"
        for j in range(5) :
            s += str(state[j + (5*i)]) + "|"
        print(s)

**Description of your heuristics and why they are admissible**

*L2 = the L made of twos, L7 = the L made of sevens*

**h1**: This heuristic finds the number of squares away L2 and L7 are from touching each other. It encourages actions that bring L2 and L7 closer together, and discourages actions where they move further apart. This is a pretty basic function that trims a lot of branches that would be absolutely useless, like moving L7 further away from L2.

**h2**: This heuristic was designed to encourage actions that actually move block out of the way of L2 and L7. It counts the number of blocks (excluding 0 squares) between L2 and L7, and if a block is standing between them and can be moved out of the way, then it should be. It also discourages moving blocks between L2 and L7, or moving them further apart. Possibly freeing up a chance for L2 and L7 to come together sooner. It doesn't necessarily force L2 and L7 to come together when they are close, it only affects other blocks standing between them.

**Admissibility**:
I believe these functions are admissible and will never overestimate the cost of reaching the goal because in h1, L2 and L7 will always be at least h1 moves away from each other. If they are sitting in opposite corners of the game board then the minimum number of moves to reach each other will be 5, which is the value of h1. With h2, the cost of L7 reaching L2 might start high but it will go down accordingly as blocks are moved out of the way. If there are no blocks in the way then the cost is 0, even if L7 or L2 still have to move to reach each other. 

Given the above code that you have written, you can now solve the puzzle.  Fill in the following method that is supposed to return a list of actions, that when applied to the initial state, will lead to a goal state:

In [35]:
def solve_puzzle(puzzle) :
    """Receives an instance of the class two_Ls and returns
    a list of actions, that when applied to an initial state, lead to a 
    goal state.
    """
    return search.astar_search(puzzle, max_h).solution()

Note that the list `[(9, 'RIGHT'), (7, 'DOWN'), (8, 'LEFT')]` comprises the first three actions found by our solution to the problem.

To test your `solve_puzzle` we will test whether the state/action sequence returned by the method is valid under the rules of the puzzle, and that they lead to a solution.  We will also test your `two_Ls` class in detail.

Answer the following:

* Run your code, and compare the number of nodes expanded by A* with the heuristics you have proposed.  
* Which heuristic worked better?  Can you explain why?  
* Do you expect DFS or BFS to yield a result in a reasonable amount of time?  Explain!

For retrieving the number of nodes expanded during the search, you may use the `InstrumentedProblem` class in `search.py`.


In [36]:
# your code for comparing heuristics
puzzle = two_Ls((1,1,1,2,2, 3,3,4,4,2, 7,5,5,6,6, 7,7,8,8,8, 0,0,0,9,0))

# Nodes expanded in A* search with heuristic h1
inst = search.InstrumentedProblem(puzzle)
search.astar_search(inst, h1)
print("Nodes expanded using h1:", inst.states)

# Nodes expanded in A* search with heuristic h2
inst = search.InstrumentedProblem(puzzle)
search.astar_search(inst, h2)
print("Nodes expanded using h2:", inst.states)

# Nodes expanded in A* search with maximum heuristics
inst = search.InstrumentedProblem(puzzle)
result = search.astar_search(inst, max_h)
print("Nodes expanded using maximum heuristics:", inst.states)

Nodes expanded using h1: 28984
Nodes expanded using h2: 4319
Nodes expanded using maximum heuristics: 4311


**Results**:

Nodes expanded using h1: 28984

Nodes expanded using h2: 4319

Nodes expanded using maximum heuristics: 4311

**Explanation**:

h2 worked better than h1 I believe because it affected more blocks. h1 only tried to get L2 and L7 to move closer together on their own, but h2 actually moved all blocks to some extent. h2 had a much greater probability of knowing which of the possible blocks that could move, should be moved, and in which direction. h1 would never know that it had to move a block out of the way for L7 to get to L2, it would try them all. But h2 doesn't know when to move L7 toward L2 because there might be the same number of 0 squares between them when it moves.

I expect DFS to work much better because going out and trying every single possible action of a node seems like a huge waste of time and defeats the point of heuristics, which is to build a path down a tree and not take unnecessary turns going branch to branch.

## Grading

Your notebook will be run and graded automatically.  So make sure your code satisfies the specified API, i.e. correct naming and behavior of your methods and classes.  In addition, the TAs will read the description of the heuristics.  

Grading Rubric:

60 pts:  Correctness of your code for solving the puzzle.

20 pts:  The proposed heuristics make sense, are distinct, and their admissibility is discussed in a convincing manner.

20 pts:  Comparison of the heuristics and discussion of your results